# KNN woe


## Adatok betöltése

In [171]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bc050f45180b4c1bb8886542b85a43fe = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6akYbYHItNWmuiZPpgS8f0DnZNTVPSN3hYgF_EVLfHhd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_input = pd.read_csv(body, sep=';',decimal=',')
train_input.head(10)


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,-0.893508
2,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,-0.224232
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.224232
5,0,-0.029802,-0.180531,-0.210588,-0.365448,-0.883030,0.585450
6,0,0.276928,-0.180531,0.728017,0.603264,0.232891,0.585450
7,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
8,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,-0.224232
9,0,-0.029802,-0.180531,-0.210588,-0.365448,0.232891,0.585450


In [172]:

body = client_bc050f45180b4c1bb8886542b85a43fe.get_object(Bucket='szakdolgozat-donotdelete-pr-nxru0g4tgksnkv',Key='WOE_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_input = pd.read_csv(body, sep=';',decimal=',')
test_input.head()


,DPD90_12m,customer_type,product_type,onero,age_of_asset (honap),sales_channel,Nace risk tier
0,0,-0.029802,-0.180531,0.010955,0.603264,0.232891,0.585450
1,0,-0.029802,-0.180531,-0.210588,-1.386171,-0.883030,0.585450
2,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
3,0,-0.029802,-0.180531,0.728017,0.603264,0.232891,0.585450
4,0,-0.029802,-0.180531,-0.210588,0.603264,0.232891,-0.893508


In [173]:
train_input.shape

(7339, 7)

In [174]:
test_input.shape

(3083, 7)

## Adatok előkészítése

In [175]:
import numpy as np

In [176]:
X_train = np.asarray(train_input[['onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier', 'customer_type', 'product_type']])
X_train[0:1]

array([[ 0.01095466,  0.6032641 ,  0.23289082,  0.58545026, -0.02980206,
        -0.18053093]])

In [177]:
X_test = np.asarray(test_input[['onero', 'age_of_asset (honap)','sales_channel', 'Nace risk tier', 'customer_type', 'product_type']])
X_test[0:1]

array([[ 0.01095466,  0.6032641 ,  0.23289082,  0.58545026, -0.02980206,
        -0.18053093]])

In [178]:
y_train = np.asarray(train_input['DPD90_12m'])
y_train [0:5]

array([0, 0, 0, 0, 0])

In [179]:
y_test = np.asarray(test_input['DPD90_12m'])
y_test [0:5]

array([0, 0, 0, 0, 0])

## Modellezés

In [215]:
from sklearn.neighbors import KNeighborsClassifier
LR = KNeighborsClassifier(n_neighbors=3500).fit(X_train,y_train)
LR

KNeighborsClassifier(n_neighbors=3500)

In [216]:
yhat_test = LR.predict_proba(X_test)
yhat_test

array([[0.99228571, 0.00771429],
       [0.96714286, 0.03285714],
       [0.99285714, 0.00714286],
       ...,
       [0.96457143, 0.03542857],
       [0.95771429, 0.04228571],
       [0.98971429, 0.01028571]])

In [217]:
yhat_train = LR.predict_proba(X_train)
yhat_train

array([[0.99228571, 0.00771429],
       [0.95857143, 0.04142857],
       [0.97285714, 0.02714286],
       ...,
       [0.98971429, 0.01028571],
       [0.99285714, 0.00714286],
       [0.96285714, 0.03714286]])

In [218]:
tech, y_predicted_train = zip(*yhat_train)
y_predicted_train[0:5]

(0.007714285714285714,
 0.041428571428571426,
 0.027142857142857142,
 0.013142857142857144,
 0.018)

In [219]:
tech, y_predicted_test = zip(*yhat_test)
y_predicted_test[0:5]

(0.007714285714285714,
 0.032857142857142856,
 0.007142857142857143,
 0.007142857142857143,
 0.02057142857142857)

## Modell ereje

In [220]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_train, y_predicted_train)
gini=2*roc-1
gini

0.5225754673018175

In [221]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test, y_predicted_test)
gini=2*roc-1
gini

0.5352973706267119